In [1]:
import pandas as pd
import numpy as np
import os
import shapefile
import ast
from pyproj import Transformer

In [2]:
#https://stackoverflow.com/questions/55112771/read-shapefiles-into-dataframe
def read_shapefile(sf_shape):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' 
    column holding the geometry information. This uses the pyshp
    package
    """

    fields = [x[0] for x in sf_shape.fields][1:]
    records = [y[:] for y in sf_shape.records()]
    #records = sf_shape.records()
    shps = [s.points for s in sf_shape.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df

## Finding the community centers

https://bostonopendata-boston.opendata.arcgis.com/datasets/acedd06b39854088b0f2fefccffebf84_0?geometry=-71.607%2C42.234%2C-70.564%2C42.412

In [3]:
local_data_path = "../../dataset_ignore/community_centers/"

In [4]:
local_data_path

'../../dataset_ignore/community_centers/'

In [5]:
shp_path = os.listdir(local_data_path)[3]
shp_path

'20a369e6-57fe-45f1-b3d9-381a07ca8b5f2020328-1-1eby637.wzx.shp'

In [6]:
shp_file = shapefile.Reader(local_data_path +  shp_path)
df = read_shapefile(shp_file)
df

,FID,OBJECTID,SITE,PHONE,FAX,STREET,NEIGH,ZIP,coords
0,1,0.0,Pino,635-5120,635-1874,86 Boardman St.,East Boston,2128,"[[-71.0108269837395, 42.3888817140619]]"
1,2,0.0,Paris Street Pool*,635-1410,635-5079,113 Paris St.,East Boston,2128,"[[-71.0368919163415, 42.3724271511922]]"
2,3,0.0,Paris Street,635-5125,635-5079,112 Paris St.,East Boston,2128,"[[-71.0375220248402, 42.3728816312829]]"
3,4,0.0,Golden Age Center,635-5175,635-5647,382 Main St.,Charlestown,2129,"[[-71.0699168142103, 42.3797690168046]]"
4,5,0.0,Charlestown*,635-5169,635-5084,255 Medford St.,Charlestown,2129,"[[-71.0609507192311, 42.380995303203]]"
5,6,0.0,Nazzaro,635-5166,635-5053,30 N. Bennet St.,Boston,2113,"[[-71.0544968810165, 42.3652996796642]]"
6,7,0.0,Quincy*,635-5129,635-5132,885 Washington St,Boston,2111,"[[-71.0649617695614, 42.3479492763056]]"
7,8,0.0,Condon*,635-5100,635-5103,"200 ""D"" St.",South Boston,2127,"[[-71.0525266409767, 42.338151479745]]"
8,9,0.0,Tynan,635-5110,635-5065,650 E. Fourth St.,South Boston,2127,"[[-71.0389136775342, 42.3346334388507]]"
9,10,0.0,Curley,635-5104,635-1259,1663 Columbia Rd.,South Boston,2127,"[[-71.0354629468184, 42.3292072485415]]"


For some reason, there's a * at the end of some of the names in the community center dataset, let's remove that *

In [7]:
df['SITE'] = df.apply(lambda x:x['SITE'][0:-1] if x['SITE'].find('*') != -1 else x['SITE'], axis = 1)
df

,FID,OBJECTID,SITE,PHONE,FAX,STREET,NEIGH,ZIP,coords
0,1,0.0,Pino,635-5120,635-1874,86 Boardman St.,East Boston,2128,"[[-71.0108269837395, 42.3888817140619]]"
1,2,0.0,Paris Street Pool,635-1410,635-5079,113 Paris St.,East Boston,2128,"[[-71.0368919163415, 42.3724271511922]]"
2,3,0.0,Paris Street,635-5125,635-5079,112 Paris St.,East Boston,2128,"[[-71.0375220248402, 42.3728816312829]]"
3,4,0.0,Golden Age Center,635-5175,635-5647,382 Main St.,Charlestown,2129,"[[-71.0699168142103, 42.3797690168046]]"
4,5,0.0,Charlestown,635-5169,635-5084,255 Medford St.,Charlestown,2129,"[[-71.0609507192311, 42.380995303203]]"
5,6,0.0,Nazzaro,635-5166,635-5053,30 N. Bennet St.,Boston,2113,"[[-71.0544968810165, 42.3652996796642]]"
6,7,0.0,Quincy,635-5129,635-5132,885 Washington St,Boston,2111,"[[-71.0649617695614, 42.3479492763056]]"
7,8,0.0,Condon,635-5100,635-5103,"200 ""D"" St.",South Boston,2127,"[[-71.0525266409767, 42.338151479745]]"
8,9,0.0,Tynan,635-5110,635-5065,650 E. Fourth St.,South Boston,2127,"[[-71.0389136775342, 42.3346334388507]]"
9,10,0.0,Curley,635-5104,635-1259,1663 Columbia Rd.,South Boston,2127,"[[-71.0354629468184, 42.3292072485415]]"


Reading in the business csv, want to make sure there are no redundancies

In [8]:
local_social = "../../datasets_clean/"

In [9]:
os.listdir(local_social)[1]

'businesses_sanitized.csv'

In [10]:
old_soc = pd.read_csv(local_social + os.listdir(local_social)[1])
old_soc

,Unnamed: 0,name,zip,lon,lat,category
0,0,Franklin Cafe,2118,-71.068979,42.342915,food
1,1,South End Buttery,2118,-71.070373,42.342101,food
2,2,J.J. Foley's Cafe,2118,-71.065280,42.343664,food
3,3,Cathedral Station,2118,-71.066770,42.342806,food
4,4,Dunkin',2118,-71.065905,42.343730,food
...,...,...,...,...,...,...
70619,70619,AXA Advisors,2459,NaN,NaN,commercial
70620,70620,Barry D Lang MD & Associates,2459,NaN,NaN,commercial
70621,70621,"Doggone It, Inc.",2459,NaN,NaN,commercial
70622,70622,Fetch,2459,NaN,NaN,commercial


## Cleaning community center dataset

In [11]:
df = df[['SITE','STREET','ZIP','coords']]
df['category'] = 'social'
df

<ipython-input-11-f2761cf659e2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = 'social'


,SITE,STREET,ZIP,coords,category
0,Pino,86 Boardman St.,2128,"[[-71.0108269837395, 42.3888817140619]]",social
1,Paris Street Pool,113 Paris St.,2128,"[[-71.0368919163415, 42.3724271511922]]",social
2,Paris Street,112 Paris St.,2128,"[[-71.0375220248402, 42.3728816312829]]",social
3,Golden Age Center,382 Main St.,2129,"[[-71.0699168142103, 42.3797690168046]]",social
4,Charlestown,255 Medford St.,2129,"[[-71.0609507192311, 42.380995303203]]",social
5,Nazzaro,30 N. Bennet St.,2113,"[[-71.0544968810165, 42.3652996796642]]",social
6,Quincy,885 Washington St,2111,"[[-71.0649617695614, 42.3479492763056]]",social
7,Condon,"200 ""D"" St.",2127,"[[-71.0525266409767, 42.338151479745]]",social
8,Tynan,650 E. Fourth St.,2127,"[[-71.0389136775342, 42.3346334388507]]",social
9,Curley,1663 Columbia Rd.,2127,"[[-71.0354629468184, 42.3292072485415]]",social


In [12]:
df['lower'] = df['SITE'].str.casefold()
old_soc['lower'] = old_soc['name'].str.casefold()

df = df[~df.lower.isin(old_soc.lower)]

community_centers = df.drop(columns = 'lower')
community_centers

<ipython-input-12-b979db8d2b68>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lower'] = df['SITE'].str.casefold()


,SITE,STREET,ZIP,coords,category
0,Pino,86 Boardman St.,2128,"[[-71.0108269837395, 42.3888817140619]]",social
2,Paris Street,112 Paris St.,2128,"[[-71.0375220248402, 42.3728816312829]]",social
5,Nazzaro,30 N. Bennet St.,2113,"[[-71.0544968810165, 42.3652996796642]]",social
6,Quincy,885 Washington St,2111,"[[-71.0649617695614, 42.3479492763056]]",social
7,Condon,"200 ""D"" St.",2127,"[[-71.0525266409767, 42.338151479745]]",social
8,Tynan,650 E. Fourth St.,2127,"[[-71.0389136775342, 42.3346334388507]]",social
9,Curley,1663 Columbia Rd.,2127,"[[-71.0354629468184, 42.3292072485415]]",social
10,Vine Street,339 Dudley Street,2119,"[[-71.0767147735931, 42.3267818083174]]",social
12,Rec. Division @ Madison Park,55 Malcom X Blvd,2119,"[[-71.0892278686055, 42.3316860264015]]",social
13,Tobin,1481 Tremont St.,2119,"[[-71.0981616896089, 42.3324019255967]]",social


## Cleaning religion

In [13]:
local_data_path = "../../dataset_ignore/places_of_worship/"

In [14]:
shp_path = os.listdir(local_data_path)[6]
shp_path

'Places_of_Worship.shp'

In [15]:
shp_file = shapefile.Reader(local_data_path +  shp_path)
df = read_shapefile(shp_file)

In [16]:
df = df[df['GEOG_CITY'] == 'Boston']

In [17]:
df = df[['NAME','ADDRESS','ZIPCODE','coords']]
df['category'] = 'social'
df

,NAME,ADDRESS,ZIPCODE,coords,category
2,Islamic Society of Boston Cultural Center,100 Malcolm X Boulevard,02120,"[[233512.96549999714, 897904.6944999993]]",social
8,Masjid Al-Noor,28 Circuit Street,02119,"[[234133.40370000154, 897057.6695999987]]",social
11,Masjidun Lil Hamidulillah,724 Shawmut Avenue,02119,"[[234346.56319999695, 898114.9417999983]]",social
13,Allston Brighton Islamic Center,15 North Beacon Street,02134,"[[229852.79240000248, 900481.2523999996]]",social
17,Masjid Al-Quran,35 Intervale Street,02125,"[[234597.54389999807, 895756.3563000001]]",social
...,...,...,...,...,...
1963,The Community Church of Boston,565 Boylston Street,02116,"[[234903.56340000033, 900105.0887999982]]",social
2024,First Church in Charlestown,10 Green Street,02129-0203,"[[235927.3343999982, 902910.5007999986]]",social
2026,Hope Central Church,85 Seaverns Avenue,02130-2866,"[[232164.33110000193, 895676.2529000007]]",social
2027,Roslindale Congregational Church,25 Cummins Hwy,02131-2523,"[[230720.63599999994, 892866.2316999994]]",social


In [18]:
df['lower'] = df['NAME'].str.casefold()
old_soc['lower'] = old_soc['name'].str.casefold()
df = df[~df.lower.isin(old_soc.lower)]
religion = df.drop(columns = 'lower')
religion

,NAME,ADDRESS,ZIPCODE,coords,category
8,Masjid Al-Noor,28 Circuit Street,02119,"[[234133.40370000154, 897057.6695999987]]",social
11,Masjidun Lil Hamidulillah,724 Shawmut Avenue,02119,"[[234346.56319999695, 898114.9417999983]]",social
18,Yousuf Masjid,186 Chestnut Hill Avenue,02135,"[[228469.74549999833, 898993.511500001]]",social
49,Chabad House,491 Commonwealth Avenue,02215,"[[233425.1704000011, 899954.1667999998]]",social
73,Temple Hillel Bnai Torah,120 Corey Street,02132,"[[228290.81329999864, 893083.7446999997]]",social
...,...,...,...,...,...
1801,Pilgrim Church of Dorchester,540 Columbia Road,02125-2323,"[[235726.3348999992, 896258.0102999993]]",social
1849,First Church in Jamaica Plain UU,6 Eliot Street,02130,"[[231639.21299999952, 895576.3126000017]]",social
1853,"Church of the Covenant, Federated",67 Newbury Street,02116-3090,"[[235082.89940000325, 900269.185899999]]",social
1963,The Community Church of Boston,565 Boylston Street,02116,"[[234903.56340000033, 900105.0887999982]]",social


In [19]:
community_centers.rename(columns = {'ZIP':'ZIPCODE','SITE':'NAME','STREET':'ADDRESS'}, inplace = True)

In [20]:
socialdf = pd.concat((religion,community_centers))
socialdf

,NAME,ADDRESS,ZIPCODE,coords,category
8,Masjid Al-Noor,28 Circuit Street,02119,"[[234133.40370000154, 897057.6695999987]]",social
11,Masjidun Lil Hamidulillah,724 Shawmut Avenue,02119,"[[234346.56319999695, 898114.9417999983]]",social
18,Yousuf Masjid,186 Chestnut Hill Avenue,02135,"[[228469.74549999833, 898993.511500001]]",social
49,Chabad House,491 Commonwealth Avenue,02215,"[[233425.1704000011, 899954.1667999998]]",social
73,Temple Hillel Bnai Torah,120 Corey Street,02132,"[[228290.81329999864, 893083.7446999997]]",social
...,...,...,...,...,...
26,Menino,125 Brookway Rd.,2131,"[[-71.1201491289981, 42.2923882465749]]",social
27,Roche,1716 Centre St.,2132,"[[-71.1493061558172, 42.2869359902307]]",social
29,Ohrenberger,175 W. Boundary R,2132,"[[-71.1494662657657, 42.2608343327245]]",social
30,Jackson/Mann,500 Cambridge St.,2134,"[[-71.1375007708588, 42.3529092061974]]",social


In [53]:
raw = socialdf
raw

,NAME,ADDRESS,ZIPCODE,coords,category,east,north
8,Masjid Al-Noor,28 Circuit Street,02119,"[[234133.40370000154, 897057.6695999987]]",social,234133.403700,897057.669600
11,Masjidun Lil Hamidulillah,724 Shawmut Avenue,02119,"[[234346.56319999695, 898114.9417999983]]",social,234346.563200,898114.941800
18,Yousuf Masjid,186 Chestnut Hill Avenue,02135,"[[228469.74549999833, 898993.511500001]]",social,228469.745500,898993.511500
49,Chabad House,491 Commonwealth Avenue,02215,"[[233425.1704000011, 899954.1667999998]]",social,233425.170400,899954.166800
73,Temple Hillel Bnai Torah,120 Corey Street,02132,"[[228290.81329999864, 893083.7446999997]]",social,228290.813300,893083.744700
...,...,...,...,...,...,...,...
26,Menino,125 Brookway Rd.,2131,"[[-71.1201491289981, 42.2923882465749]]",social,-71.120149,42.292388
27,Roche,1716 Centre St.,2132,"[[-71.1493061558172, 42.2869359902307]]",social,-71.149306,42.286936
29,Ohrenberger,175 W. Boundary R,2132,"[[-71.1494662657657, 42.2608343327245]]",social,-71.149466,42.260834
30,Jackson/Mann,500 Cambridge St.,2134,"[[-71.1375007708588, 42.3529092061974]]",social,-71.137501,42.352909


In [54]:
raw = raw.reset_index(drop = True)
raw

,NAME,ADDRESS,ZIPCODE,coords,category,east,north
0,Masjid Al-Noor,28 Circuit Street,02119,"[[234133.40370000154, 897057.6695999987]]",social,234133.403700,897057.669600
1,Masjidun Lil Hamidulillah,724 Shawmut Avenue,02119,"[[234346.56319999695, 898114.9417999983]]",social,234346.563200,898114.941800
2,Yousuf Masjid,186 Chestnut Hill Avenue,02135,"[[228469.74549999833, 898993.511500001]]",social,228469.745500,898993.511500
3,Chabad House,491 Commonwealth Avenue,02215,"[[233425.1704000011, 899954.1667999998]]",social,233425.170400,899954.166800
4,Temple Hillel Bnai Torah,120 Corey Street,02132,"[[228290.81329999864, 893083.7446999997]]",social,228290.813300,893083.744700
...,...,...,...,...,...,...,...
130,Menino,125 Brookway Rd.,2131,"[[-71.1201491289981, 42.2923882465749]]",social,-71.120149,42.292388
131,Roche,1716 Centre St.,2132,"[[-71.1493061558172, 42.2869359902307]]",social,-71.149306,42.286936
132,Ohrenberger,175 W. Boundary R,2132,"[[-71.1494662657657, 42.2608343327245]]",social,-71.149466,42.260834
133,Jackson/Mann,500 Cambridge St.,2134,"[[-71.1375007708588, 42.3529092061974]]",social,-71.137501,42.352909


In [55]:
# back to list
raw['east'] = raw['coords'].apply(lambda x: x[0][0])
raw['north'] = raw['coords'].apply(lambda x: x[0][1])


parcels = raw[['NAME', 'ADDRESS', 'ZIPCODE', 'east', 'north', 'category']]
parcels.columns = ['name', 'address', 'zip', 'east', 'north', 'category']
parcels = parcels.reset_index(drop = True)

transformer = Transformer.from_crs(crs_from = 'epsg:2805',
                                   crs_to = 'epsg:4326')

# lists of east/north and initial for lat/lon
east = list(parcels.east)
north = list(parcels.north)
latitudes = []
longitudes = []

# saving a couple entries for testing
test = parcels.loc[0:2,:].copy(deep = True).reset_index(drop = True)

# transforming
for i in range(len(east)):
    new = transformer.transform(east[i],north[i])
    latitudes.append(new[0])
    longitudes.append(new[1])

# updating dataframe
parcels.east = latitudes
parcels.north = longitudes



# checking first couple entries
test1 = transformer.transform(test.east[0],test.north[0])
test2 = transformer.transform(test.east[1],test.north[1])

if test1 == (parcels.east[0],parcels.north[0]) and \
    test2 == (parcels.east[1],parcels.north[1]):
        parcels.columns = ['name', 'address', 'zip', 'lat', 'lon', 'category']
        print('converted.')
        
else:
    print('improper conversion.')
    

converted.


In [57]:
parcels.to_csv('../../datasets_clean/worship_community_latlon.csv')